<table align="left">
  <tr>
    <td><img src="Museum.png" alt="ice cream truck in front of Louvre" width="120"/></td>
    <td align="left"><h1>Lesson 5: Building a Deep Research-like agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Set up and test search tools

In [1]:
from tavily import TavilyClient
from smolagents import tool, Tool
import os
from dotenv import load_dotenv

load_dotenv(override=True) # load variables from local .env file

@tool
def web_search(query: str) -> str:
    """Searches the web for your query.

    Args:
        query: Your query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(query)
    return str(response["results"])


class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, 40000)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

print(web_search("What is the melting temperature of vanilla ice cream in °C?"))

[{'url': 'https://www.quora.com/At-what-temperature-does-ice-cream-melt-Is-it-higher-or-lower-than-water', 'title': 'At what temperature does ice cream melt? Is it higher or lower than ...', 'content': 'Ice Cream will begin melting once when warmer than -15˚C. Shops generally store ice cream in their display freezers at -18˚C, whereas most', 'score': 0.6077801, 'raw_content': None}, {'url': 'http://scienceline.ucsb.edu/getkey.php?key=6870', 'title': 'UCSB Science Line', 'content': 'Hot ice cream will be a liquid (melted) at low temperature, and will solidify as the temperature is increased. The transition temperature, technically called the gelling temperature, is around 50-70 degrees Celsius (approximately 120 - 160 degrees Fahrenheit.)', 'score': 0.5128152, 'raw_content': None}, {'url': 'https://www.icecreamscience.com/blog/ice-cream-melt', 'title': 'Why does ice cream melt?', 'content': 'Crystallisation of fat globules takes place during the ageing stage where the ice cream mix is k

## Set up an agent

In [3]:
from PIL import Image

from smolagents import CodeAgent, OpenAIServerModel

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

code_model = "gpt-4.1-mini"

model = OpenAIServerModel(
    #"gpt-4o",
    code_model,  #updated model
    max_completion_tokens=8096,
)

<div style="font-size: 20px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> In the notebook, we reduced the museum count from 10 to 3 to reduce runtime and cost.  
  This will reduce the number of ice cream trucks you will need to buy to start your business!
</div>


In [4]:
#changed to 2024 to avoid decaying internet content
request_museums = """
Could you give me a sorted list of the top 3 museums in the world in 2024,
along with their visitor count (in millions) that year, and the approximate daily temperature
in July at their location ?"""

In [5]:
agent = CodeAgent(
    model=model,
    tools=[web_search, VisitWebpageTool()],
    max_steps=10
)
agent.logger.console.width=66
result = agent.run(request_museums)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  museums_2024 = web_search(query="top museums in the world 2024  
  by visitor count")                                              
  print(museums_2024)                                             
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'url': 
'https://www.theartnewspaper.com/2025/04/01/the-worlds-most-visite
d-museums-2024-', 'title': "Exclusive | The world's most-visited 
museums 2024", 'content': 'Meanwhile, the Natural History Museum, 
not included in our survey, received 6.3 million and had a record 
number of visitors in its 2023-24 financial year. Of course, the 
very existence of the National Museum of African American History 
and Culture could be argued as political—and, with its consistent 
1.6 million visitors both last year and in 2023, it is the fourth 
most popular art museum in the country, and the second most 
visited in the nation’s capital. In third place, after the 
National Gallery of Art, is the Museum of Modern Art (MoMA) with 
2.7 million visitors (down 6% on 2023).', 'score': 0.98508, 
'raw_content': None}, {'url': 
'https://www.cntravellerme.com/story/the-most-visited-museums-in-t
he-world-and-whats-on-this-year', 'title': "The most visited 
museums in the world, and what's on this year", 'content': 'Musée 
du Louvre, Paris: 8,737,050 visitors in 2024 · Vatican Museums, 
Vatican City: 6,825,436 visitors in 2024 · British Museum, London:
6,479,952', 'score': 0.98182, 'raw_content': None}, {'url': 
'https://www.statista.com/statistics/246293/art-museums-by-total-a
ttendance-worldwide/', 'title': 'Most visited art museums 
worldwide 2024 - Statista', 'content': 'In 2024, the Louvre Museum
in Paris, France, was the most visited art museum worldwide, 
recording approximately *** million visitors.', 'score': 0.97794, 
'raw_content': None}, {'url': 
'https://www.wanderlustmagazine.com/news/most-visited-museums-2024
/', 'title': 'These were the most-visited museums around the world
in 2024', 'content': "In second place, with 6.8 million visitors 
was the Vatican Museums, followed closely by London's British 
Museum with 6.4 million guests. The", 'score': 0.97411, 
'raw_content': None}, {'url': 
'https://www.facebook.com/theartnewspaper/photos/exclusive-the-wor
lds-most-visited-museums-2024-normality-returnsfor-somea-new-mu/10
61826645983623/', 'title': "The Art - Exclusive | The world's most
visited museums 2024 ...", 'content': "Exclusive | The world's 
most visited museums 2024: normality returns—for some A new museum
in Shanghai leaps into our top ten and European", 'score': 
0.96753, 'raw_content': None}]

Out: None

[Step 1: Duration 1.49 seconds| Input tokens: 2,097 | Output 
tokens: 134]

...

[Step 2: Duration 7.51 seconds| Input tokens: 5,043 | Output 
tokens: 318]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  top_museums_2024 = [                                            
      {"name": "Louvre Museum", "location": "Paris",              
  "visitor_count_millions": 8.7, "jul_temp_f": 75},               
      {"name": "Vatican Museums", "location": "Vatican City",     
  "visitor_count_millions": 6.8, "jul_temp_f": 86.5},             
      {"name": "British Museum", "location": "London",            
  "visitor_count_millions": 6.4, "jul_temp_f": 69},               
  ]                                                               
                                                                  
  # Sort descending by visitor count (already sorted from         
  collected data)                                                 
  sorted_museums = sorted(top_museums_2024, key=lambda x:         
  x["visitor_count_millions"], reverse=True)                      
                                                                  
  final_answer(sorted_museums)                                    
 ────────────────────────────────────────────────────────────────

Out - Final answer: [{'name': 'Louvre Museum', 'location': 
'Paris', 'visitor_count_millions': 8.7, 'jul_temp_f': 75}, 
{'name': 'Vatican Museums', 'location': 'Vatican City', 
'visitor_count_millions': 6.8, 'jul_temp_f': 86.5}, {'name': 
'British Museum', 'location': 'London', 'visitor_count_millions': 
6.4, 'jul_temp_f': 69}]

[Step 3: Duration 4.12 seconds| Input tokens: 10,117 | Output 
tokens: 603]

In [6]:
#modified code to handle case where agent doesn't return a python list.
import pandas as pd

try:
    display(pd.DataFrame(result))
except Exception as e:
    print("Could not display as DataFrame:", e)
    print(result)

,name,location,visitor_count_millions,jul_temp_f
0,Louvre Museum,Paris,8.7,75.0
1,Vatican Museums,Vatican City,6.8,86.5
2,British Museum,London,6.4,69.0


## Create a multi-agent researcher
### Start with web search agent

In [7]:
web_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,  # updated model
        max_completion_tokens=8096,
    ),
    tools=[web_search, VisitWebpageTool()],
    max_steps=10,
    name="web_agent",
    description="Runs web searches for you."
)
web_agent.logger.console.width=66

### Setup manager agent

In [8]:
from smolagents import Tool
from typing import Any
from smolagents.utils import make_image_url, encode_image_base64

def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel(
        "gpt-4o",
    )
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
        "Also, any run that invents numbers should fail."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,
        max_tokens=8096,
    ),
    tools=[],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",  #added
        "plotly.express.colors",  #added
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)
manager_agent.logger.console.width=66

In [9]:
manager_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: ['geopandas', 'plotly', 
│   'plotly.express', 'plotly.express.colors', 'shapely', 'json', 
│   'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description          ┃ Arguments            ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final     │ answer (`any`): The  │
│   │              │ answer to the given  │ final answer to the  │
│   │              │ problem.             │ problem              │
│   └──────────────┴──────────────────────┴──────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description      ┃ Arguments       ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Searches the web │ query           │
            │               │ for your query.  │ (`string`):     │
            │               │                  │ Your query      │
            │ visit_webpage │ Visits a webpage │ url (`string`): │
            │               │ at the given url │ The url of the  │
            │               │ and reads its    │ webpage to      │
            │               │ content as a     │ visit.          │
            │               │ markdown string. │                 │
            │               │ Use this to      │                 │
            │               │ browse webpages. │                 │
            │ final_answer  │ Provides a final │ answer (`any`): │
            │               │ answer to the    │ The final       │
            │               │ given problem.   │ answer to the   │
            │               │                  │ problem         │
            └───────────────┴──────────────────┴─────────────────┘

In [10]:
# added this code, remove plot file if exits.
import os
os.path.exists("saved_map.png") and os.remove("saved_map.png")

False

In [ ]:
manager_agent.run(f"""
{request_museums}

Then make me a spatial map of the world using px.scatter_map, with the biggest museums 
represented as scatter points of size depending on visitor count and color depending 
on the average temperature in July.
Save the map to saved_map.png, then return it!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour",
     color_continuous_scale=px.colors.sequential.Magma_r, size_max=15, zoom=1)
fig.show()
final_answer(fig)

Do not invent any numbers! You must only use numbers sourced from the internet.
""")

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
│ Then make me a spatial map of the world using px.scatter_map,  │
│ with the biggest museums                                       │
│ represented as scatter points of size depending on visitor     │
│ count and color depending                                      │
│ on the average temperature in July.                            │
│ Save the map to saved_map.png, then return it!                 │
│                                                                │
│ Here's an example of how to plot and return a map:             │
│ import plotly.express as px                                    │
│ df = px.data.carshare()                                        │
│ fig = px.scatter_map(df, lat="centroid_lat",                   │
│ lon="centroid_lon", text="name", color="peak_hour",            │
│      color_continuous_scale=px.colors.sequential.Magma_r,      │
│ size_max=15, zoom=1)                                           │
│ fig.show()                                                     │
│ final_answer(fig)                                              │
│                                                                │
│ Do not invent any numbers! You must only use numbers sourced   │
│ from the internet.                                             │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

────────────────────────── Initial plan ──────────────────────────
Here are the facts I know and the plan of action that I will 
follow to solve the task:
```
---
## Facts survey
### 1.1. Facts given in the task
- The task is to produce a sorted list of the top 3 museums in the
world in 2024 by visitor count (in millions).
- Also to provide the approximate daily temperature in July at the
location of each museum.
- The map must use px.scatter_map from Plotly Express.
- Museums are to be represented as scatter points sized by visitor
count and colored by average July temperature.
- The final map should be saved to "saved_map.png" and returned.
- An example snippet of code using px.scatter_map is provided for 
reference.
- We must not invent numbers, only use sourced data from the 
internet.

### 1.2. Facts to look up
- The visitor count of the top museums in the world for the year 
2024.  
  *Source:* Recent reports or statistics published online by 
museum organizations, tourism boards, or reputable data 
aggregators such as UNESCO, The Art Newspaper, or Statista.
- The geographic coordinates (latitude and longitude) of these top
museums.  
  *Source:* Official museum websites, Google Maps, or geolocation 
APIs.
- The approximate daily temperature in July at the location of 
each museum.  
  *Source:* Meteorological data from weather websites such as 
Weather.com, World Weather Online, or historical climate databases
like NOAA or Wikipedia climate data sections.
- Confirmation of museum rankings by visitor numbers specifically 
for 2024 (not earlier years).  
  *Source:* The most recent published statistics, possibly 
requiring current web search.

### 1.3. Facts to derive
- Derive the sorted order (descending) of the top 3 museums based 
on the visitor counts for 2024.
- Extract or compute approximate average daily temperature in July
(in °C or °F) for each museum’s location.
- Determine the corresponding latitudes and longitudes to place on
the map.
- Format the data into a dataframe suitable for px.scatter_map, 
including columns: latitude, longitude, visitor count, 
temperature, name.
- Choose an appropriate range and scale for marker sizes and color
mapping based on visitor counts and temperatures.
- Generate the spatial scatter map plot with Plotly Express and 
save it as "saved_map.png".
- Return the saved figure as final output.

## Plan
1. Task a web_agent to find the most recent and reliable data 
source listing the top museums by visitor count in 2024, with 
specific visitor counts in millions.
2. Task a web_agent to lookup the exact geographic coordinates 
(latitude and longitude) for each of those top 3 museums.
3. Task a web_agent to find the average daily temperature in July 
for each museum’s city/location, preferably current or recent 
climate averages.
4. Combine the collected data into a structured format to prepare 
for visualization.
5. Derive the sorted list of the top 3 museums by their visitor 
count.
6. Create a pandas dataframe including museum name, visitor count,
latitude, longitude, and July temperature.
7. Use plotly.express.px.scatter_map to create a spatial map:
   - The size of scatter points based on visitor counts.
   - Color of scatter points based on July temperature, using a 
continuous color scale.
   - Add the museum names as text labels on the scatter points.
8. Save the resulting plot to "saved_map.png".
9. Return the generated figure using final_answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will start by asking the web_agent to find the most    
recent, reliable, and detailed data about the top 3 museums       
worldwide in 2024 by visitor count, including the visitor numbers 
in millions. This is essential for the foundation of the entire   
task.                                                             
                                                                  
Code:                                                             
```py                                                             
task = (                                                          
    "Find a reliable and recent source listing the top museums in 
the world in 2024 "                                               
    "ranked by visitor count. Extract the top 3 museums, their    
visitor counts in millions, "                                     
    "and the city or location of each museum. The data must be the
most recent for 2024 "                                            
    "and should not rely on past year estimates. Provide a clear  
list with name, visitor count, and city."                         
)                                                                 
print(task)                                                       
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  task = (                                                        
      "Find a reliable and recent source listing the top museums  
  in the world in 2024 "                                          
      "ranked by visitor count. Extract the top 3 museums, their  
  visitor counts in millions, "                                   
      "and the city or location of each museum. The data must be  
  the most recent for 2024 "                                      
      "and should not rely on past year estimates. Provide a      
  clear list with name, visitor count, and city."                 
  )                                                               
  print(task)                                                     
 ────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  louvre_coords_search = web_search(query="The Louvre Museum      
  Paris latitude longitude")                                      
  vatican_coords_search = web_search(query="Vatican Museums       
  Vatican City latitude longitude")                               
  china_museum_coords_search = web_search(query="National Museum  
  of China Beijing latitude longitude")                           
  print("Louvre Museum coordinates search results:",              
  louvre_coords_search)                                           
  print("Vatican Museums coordinates search results:",            
  vatican_coords_search)                                          
  print("National Museum of China coordinates search results:",   
  china_museum_coords_search)                                     
 ────────────────────────────────────────────────────────────────

Execution logs:
Louvre Museum coordinates search results: [{'url': 
'https://www.latlong.net/place/mus-e-du-louvre-paris-france-223.ht
ml', 'title': 'Musée du Louvre, Paris, France - Latitude and 
Longitude Finder', 'content': 'The latitude of Musée du Louvre, 
Paris, France is 48.860294, and the longitude is 2.338629. Musée 
du Louvre, Paris, France is located at France country in the', 
'score': 0.9635647, 'raw_content': None}, {'url': 
'https://latitude.to/map/fr/france/museums/14/louvre-museum', 
'title': 'GPS coordinates of Louvre Museum in France. Latitude', 
'content': 'GPS coordinates of Louvre Museum in France. Latitude: 
48.8606 Longitude: 2.3376.', 'score': 0.94608605, 'raw_content': 
None}, {'url': 
'https://www.findlatitudeandlongitude.com/l/Mus%C3%A9e+du+Louvre%2
C+Paris%2C+France/1573353/', 'title': 'Musée Du Louvre, Paris, 
France - Find Latitude and Longitude', 'content': 'Musée du 
Louvre, Paris, France (N 48° 51\' 38.1564", E 2° 20\' 15.4428") 
displayed on a map.', 'score': 0.93131906, 'raw_content': None}, 
{'url': 
'https://www.coordinatesfinder.com/coordinates/115947-the-louvre',
'title': 'GPS coordinates for The Louvre - CoordinatesFinder.com',
'content': 'GPS coordinates for The Louvre · Latitude: 48.8606111.
Longitude: 2.337644 · More coordinates for The Louvre · Extra 
information about The Louvre · Distance from', 'score': 0.9106172,
'raw_content': None}, {'url': 
'https://www.coordinatesmarker.com/coordinates/501264-louvre', 
'title': 'GPS coordinates for louvre - CoordinatesFinder.com', 
'content': 'GPS coordinates for louvre · Latitude: 48.8611473. 
Longitude: 2.33802768704666 · More coordinates for louvre · Extra 
information about louvre · Distance from louvre', 'score': 
0.8803145, 'raw_content': None}]
Vatican Museums coordinates search results: [{'url': 
'https://www.latlong.net/place/vatican-museums-rome-vatican-city-2
3412.html', 'title': 'Vatican Museums, Rome, Vatican City - 
Latitude and Longitude Finder', 'content': 'The latitude of 
Vatican Museums, Rome, Vatican City is 41.906487, and the 
longitude is 12.453641. Vatican Museums, Rome, Vatican City is 
located at Vatican City', 'score': 0.96694225, 'raw_content': 
None}, {'url': 
'https://latitude.to/articles-by-country/va/holy-see-vatican-city-
state/2874/vatican-museums', 'title': 'Latitude and longitude of 
Vatican Museums', 'content': 'GPS coordinates of Vatican Museums, 
Holy See (Vatican City State). Latitude: 41.9038 Longitude: 
12.4527.', 'score': 0.96030545, 'raw_content': None}, {'url': 
'https://www.latlong.net/country/vatican-city-241.html', 'title': 
'Places in Vatican City - Latitude and Longitude Finder', 
'content': 'Vatican Museums, Rome, Vatican City (41.906487, 
12.453641); The Papal Basilica of St. Peter in the Vatican, 
Vatican City (41.901134, 12.453314); Sistine', 'score': 0.9183121,
'raw_content': None}, {'url': 
'https://www.findlatitudeandlongitude.com/l/vatican+museums/127524
9/', 'title': 'Vatican Museums - Find Latitude and Longitude', 
'content': 'Position of vatican museums (N 41° 54\' 24.642", E 12°
27\' 9.4572") on a map.', 'score': 0.91671574, 'raw_content': 
None}, {'url': 
'https://en.wikipedia.org/wiki/Module:Location_map/data/Vatican_Ci
ty', 'title': 'Module:Location map/data/Vatican City - Wikipedia',
'content': 'Module:Location map/data/Vatican City ; 41.907793. 
12.4451378 · 41.9000577 · 41°54′14″N 12°27′06″E / 41.90392535°N 
12.4518031°E / 41.90392535; 12.4518031.', 'score': 0.76659125, 
'raw_content': None}]
National Museum of China coordinates search results: [{'url': 
'https://www.latlong.net/place/national-museum-of-china-beijing-ch
ina-27232.html', 'title': 'National Museum of China, Beijing, 
China', 'content': 'The latitude of National Museum of China, 
Beijing, China is 39.905159, and the longitude is 116.400894. 
National Museum of China, Beijing, China is located at', 'score': 
0.9697425, 'raw_content': None}, {'url': 
'https://latitude.to/articles-by-country/cn/china/16414/national

[Step 1: Duration 4.31 seconds| Input tokens: 2,253 | Output 
tokens: 177]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

> Note, the map will look a bit different as we are only searching for 3 locations.

In [ ]:
fig = manager_agent.python_executor.state["fig"]
fig.update_layout(width=700, height=700)
fig.show()